In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from __future__ import print_function
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from datetime import datetime
import os

%matplotlib inline
%config InlineBackend.figure_format = 'png'
pd.set_option("max_columns",50)

In [134]:
%%time
train = pd.read_csv("../data/train.csv", index_col=0)
train = train.reset_index(drop=True)
train = train[train["is_booking"] == 1]
np.random.seed(402)
train = train.ix[np.random.choice(train.index, 100000, replace=False)]
train = train.reset_index(drop=True)

Wall time: 9min 39s


In [39]:
# dtype_set = {'site_name':np.int32, 'posa_continent':np.int32, 'user_location_country':np.int32, 'orig_destination_distance':np.int32, \
#                  'user_id':np.int32, 'is_mobile':bool, 'is_package':bool, 'channel':np.int32, 'srch_adults_cnt':np.int32, 'srch_children_cnt':np.int32, \
#                 'srch_rm_cnt':np.int32, 'srch_destination_id':np.int32, 'srch_destination_type_id':np.int32,  'is_booking':bool, \
#                 'cnt':np.int32, 'hotel_country':np.int32, 'hotel_cluster':np.int32}

# usecols_set = ['user_id', 'site_name', 'posa_continent', 'user_location_country', 'orig_destination_distance',  'is_mobile', 'is_package', 'channel', \
#                       'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id', 'is_booking', 'cnt', 'hotel_country', 'hotel_cluster']

# %%time
# train = pd.read_csv('../data/train.csv',
#                     dtype=dtype_set,
#                     usecols=usecols_set,
#                     chunksize=1000000)



In [48]:
train["num"] = 1

In [54]:
train.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,num
0,2013-05-24 21:18:49,2,3,66,331,15294,1333.8240,553218,0,0,9,2013-05-28,2013-05-29,1,0,1,12696,5,1,1,4,8,122,15,1
1,2013-08-07 11:58:51,2,3,66,321,47535,342.4031,323881,0,0,9,2013-08-18,2013-08-22,2,0,1,12189,6,1,1,2,50,637,72,1
2,2013-07-02 11:54:21,2,3,66,348,53377,3126.4955,276361,0,0,9,2013-07-15,2013-07-23,1,0,1,2758,3,1,1,6,31,1916,58,1
3,2013-11-20 17:37:30,2,3,66,315,6849,247.3951,583620,1,0,9,2013-12-28,2013-12-30,4,2,2,8267,1,1,1,2,50,675,56,1
4,2013-06-11 05:18:21,2,3,66,348,6188,149.4963,493501,0,0,9,2013-06-11,2013-06-14,1,0,1,18741,1,1,1,2,50,462,42,1


In [82]:
table = pd.pivot_table(train, values='num', index=['hotel_cluster', 'user_id'], columns=['is_booking'], aggfunc=np.sum)

In [87]:
train[train["user_id"] == 12053]

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,num
10709,2013-06-18 13:01:18,2,3,58,50,47668,NaN,12053,0,0,3,2013-08-24,2013-08-25,3,3,2,12183,6,1,1,2,50,636,0,1
43088,2013-03-14 03:42:53,2,3,58,50,47668,NaN,12053,1,0,9,2013-03-18,2013-03-19,1,1,2,12599,5,1,1,5,2,162,43,1
51027,2013-06-10 11:59:47,2,3,58,50,47668,NaN,12053,0,0,9,2013-07-10,2013-07-12,1,3,1,8291,1,1,1,2,50,191,95,1
90481,2013-09-24 06:53:03,2,3,58,50,47668,NaN,12053,0,0,9,2013-10-12,2013-10-18,2,3,2,18489,6,1,1,6,105,29,22,1


In [86]:
train = train.drop_duplicates()

In [95]:
train.groupby(['srch_destination_id',
                         'hotel_cluster'])['is_booking'].agg(['sum','count']).head()


sum  count
srch_destination_id hotel_cluster            
2                   20               1      1
4                   32               1      1
                    58               1      1
                    67               2      2
                    78               2      2

In [99]:
train[train.srch_destination_id == 4][train.hotel_cluster == 78]

C:\Users\Byeon\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,num
42157,2013-05-08 08:16:05,2,3,68,480,10322,NaN,401296,0,0,9,2013-05-13,2013-05-15,2,0,2,4,1,1,1,6,7,246,78,1
88479,2013-04-15 04:28:33,11,3,68,236,22721,NaN,419915,0,0,2,2013-04-15,2013-04-16,2,0,1,4,1,1,1,6,7,246,78,1


In [105]:
train[["is_booking","cnt"]].head()

train["ratio"] = train["is_booking"]/train["cnt"]

In [103]:
train["cnt"].value_counts() # 대부분 한번에 보고 결제해버림..! => 다른 대체물을 많이 보는편이 아님.

1    93985
2     1109
3       73
4       25
5        3
9        1
8        1
Name: cnt, dtype: int64

In [107]:
train["ratio"].value_counts()

1.000000    93985
0.500000     1109
0.333333       73
0.250000       25
0.200000        3
0.111111        1
0.125000        1
Name: ratio, dtype: int64

In [108]:
train[train.cnt == 8]

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,num,ratio
69783,2013-04-19 18:47:42,15,1,55,23,47551,NaN,1195399,0,0,5,2013-05-12,2013-05-13,1,1,1,27277,1,1,8,2,50,824,39,1,0.125


In [109]:
train[train.user_id == 1195399]

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,num,ratio
69783,2013-04-19 18:47:42,15,1,55,23,47551,NaN,1195399,0,0,5,2013-05-12,2013-05-13,1,1,1,27277,1,1,8,2,50,824,39,1,0.125


In [47]:
train.groupby("hotel_cluster").agg(sum)

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market
hotel_cluster,,,,,,,,,,,,,,,,,,,,
0,4200,1921,55803,181629,17910853,9.208420e+05,386495148,67,249,4139,1322,355,732,7584651,1803,660,669,1619,33559,353795
1,10533,4420,142975,448962,43555785,1.423071e+06,940816207,121,615,9874,3147,414,1860,14961930,3904,1550,1589,3100,77500,973400
2,18650,3661,126898,520246,42613272,2.243519e+06,910474967,120,262,9464,2630,443,1645,21380260,3953,1468,1491,6020,134093,601562
3,7308,1444,48828,182622,16216345,8.930333e+05,362673259,43,65,3423,1068,211,639,10030492,1678,576,591,2116,53321,399338
4,10509,3315,115939,343064,34405344,9.754157e+05,685113256,106,105,7637,2147,346,1307,17473062,4146,1156,1168,2539,89833,714910
5,17705,3502,120987,456694,38798773,1.886778e+06,893606082,112,180,8999,2483,509,1591,22423335,4264,1405,1423,5538,137517,673527
6,13880,4008,135951,457290,39649857,1.248556e+06,863889090,128,118,9448,2617,435,1614,22471204,4479,1441,1456,3886,116254,855829
7,8685,2965,92624,335865,30121395,7.106055e+05,662099105,109,50,7082,1949,370,1229,18011030,3105,1065,1075,2659,72704,816135
8,8240,1749,59434,227224,19869107,1.359002e+06,426086213,52,68,4363,1363,315,782,10650146,1918,694,715,2771,58489,550224


In [88]:
train1 = pd.read_csv('../data/train_2013.csv',
                    dtype={'is_booking':bool,'srch_destination_id':np.int32, 'hotel_cluster':np.int32},
                    usecols=['srch_destination_id','is_booking','hotel_cluster'],
                    chunksize=1000000)

# dtype을 설정하고(bool, np.int32) chunksize로 끊어서 하면 더 빠르게 데이터를 처리할 수 있음!!


aggs = []
print('-'*38)
for chunk in train1:
    agg = chunk.groupby(['srch_destination_id',
                         'hotel_cluster'])['is_booking'].agg(['sum','count'])
    agg.reset_index(inplace=True)
    aggs.append(agg)
    print('.',end='')
print('')
aggs = pd.concat(aggs, axis=0)

--------------------------------------
............


In [89]:
CLICK_WEIGHT = 0.05
agg = aggs.groupby(['srch_destination_id','hotel_cluster']).sum().reset_index()
agg.head()

,srch_destination_id,hotel_cluster,sum,count
0,0,3,0.0,2
1,1,20,1.0,10
2,1,30,0.0,5
3,1,57,0.0,1
4,1,60,0.0,6


In [90]:
agg['count'] -= agg['sum']
# sum은 실제 booking과 count를 합친 것..!
agg = agg.rename(columns={'sum':'bookings','count':'clicks'})
agg['relevance'] = agg['bookings'] + CLICK_WEIGHT * agg['clicks']
agg.head()

,srch_destination_id,hotel_cluster,bookings,clicks,relevance
0,0,3,0.0,2.0,0.10
1,1,20,1.0,9.0,1.45
2,1,30,0.0,5.0,0.25
3,1,57,0.0,1.0,0.05
4,1,60,0.0,6.0,0.30


In [91]:
def most_popular(group, n_max=5):
    relevance = group['relevance'].values
    hotel_cluster = group['hotel_cluster'].values
    most_popular = hotel_cluster[np.argsort(relevance)[::-1]][:n_max]
    return np.array_str(most_popular)[1:-1] # remove square brackets

most_pop = agg.groupby(['srch_destination_id']).apply(most_popular)
most_pop = pd.DataFrame(most_pop).rename(columns={0:'hotel_cluster'})
most_pop.head()


,hotel_cluster
srch_destination_id,
0,3
1,20 60 30 57
2,20 30
3,60 53
4,32 78 81 25 30


In [92]:
most_pop.tail()

,hotel_cluster
srch_destination_id,
62927,28 91 77 16 15
62929,99 28 6 7 5
62938,82
62943,89 3
62962,94


In [93]:
train

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,num
0,2013-05-24 21:18:49,2,3,66,331,15294,1333.8240,553218,0,0,9,2013-05-28,2013-05-29,1,0,1,12696,5,1,1,4,8,122,15,1
1,2013-08-07 11:58:51,2,3,66,321,47535,342.4031,323881,0,0,9,2013-08-18,2013-08-22,2,0,1,12189,6,1,1,2,50,637,72,1
2,2013-07-02 11:54:21,2,3,66,348,53377,3126.4955,276361,0,0,9,2013-07-15,2013-07-23,1,0,1,2758,3,1,1,6,31,1916,58,1
3,2013-11-20 17:37:30,2,3,66,315,6849,247.3951,583620,1,0,9,2013-12-28,2013-12-30,4,2,2,8267,1,1,1,2,50,675,56,1
4,2013-06-11 05:18:21,2,3,66,348,6188,149.4963,493501,0,0,9,2013-06-11,2013-06-14,1,0,1,18741,1,1,1,2,50,462,42,1
5,2013-07-26 09:26:49,24,2,3,51,41641,NaN,1162102,0,0,9,2013-09-12,2013-09-18,4,0,2,669,3,1,1,2,50,212,0,1
6,2013-11-13 17:21:53,2,3,194,38,42328,NaN,579419,0,0,9,2013-12-31,2014-01-01,3,0,1,14908,1,1,1,3,88,1032,96,1
7,2013-02-13 10:53:04,2,3,66,356,34146,2726.9094,423168,0,0,9,2013-02-19,2013-02-28,1,0,1,11973,1,1,1,2,198,1228,18,1
8,2013-05-01 11:35:38,2,3,66,447,902,371.3049,187763,0,0,9,2013-05-11,2013-05-14,1,0,1,12194,6,1,1,2,50,656,95,1
9,2013-02-07 19:40:07,2,3,66,174,5938,856.9267,1016357,1,0,9,2013-02-27,2013-03-01,2,0,1,12004,1,1,1,2,50,480,91,1


In [112]:
train["hotel_cluster"]

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,num,ratio
0,2013-05-24 21:18:49,2,3,66,331,15294,1333.8240,553218,0,0,9,2013-05-28,2013-05-29,1,0,1,12696,5,1,1,4,8,122,15,1,1.0
1,2013-08-07 11:58:51,2,3,66,321,47535,342.4031,323881,0,0,9,2013-08-18,2013-08-22,2,0,1,12189,6,1,1,2,50,637,72,1,1.0
2,2013-07-02 11:54:21,2,3,66,348,53377,3126.4955,276361,0,0,9,2013-07-15,2013-07-23,1,0,1,2758,3,1,1,6,31,1916,58,1,1.0
3,2013-11-20 17:37:30,2,3,66,315,6849,247.3951,583620,1,0,9,2013-12-28,2013-12-30,4,2,2,8267,1,1,1,2,50,675,56,1,1.0
4,2013-06-11 05:18:21,2,3,66,348,6188,149.4963,493501,0,0,9,2013-06-11,2013-06-14,1,0,1,18741,1,1,1,2,50,462,42,1,1.0
5,2013-07-26 09:26:49,24,2,3,51,41641,NaN,1162102,0,0,9,2013-09-12,2013-09-18,4,0,2,669,3,1,1,2,50,212,0,1,1.0
6,2013-11-13 17:21:53,2,3,194,38,42328,NaN,579419,0,0,9,2013-12-31,2014-01-01,3,0,1,14908,1,1,1,3,88,1032,96,1,1.0
7,2013-02-13 10:53:04,2,3,66,356,34146,2726.9094,423168,0,0,9,2013-02-19,2013-02-28,1,0,1,11973,1,1,1,2,198,1228,18,1,1.0
8,2013-05-01 11:35:38,2,3,66,447,902,371.3049,187763,0,0,9,2013-05-11,2013-05-14,1,0,1,12194,6,1,1,2,50,656,95,1,1.0
9,2013-02-07 19:40:07,2,3,66,174,5938,856.9267,1016357,1,0,9,2013-02-27,2013-03-01,2,0,1,12004,1,1,1,2,50,480,91,1,1.0


In [129]:
delete_list = ["posa_continent", "user_location_country", "user_location_region", "user_location_city", "is_mobile", "channel"]

In [130]:
train = train.drop(delete_list, axis=1)

In [119]:
train.sort_values("hotel_cluster")

,date_time,site_name,orig_destination_distance,user_id,is_package,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,num,ratio
54795,2013-02-06 13:29:43,2,2512.1343,415484,1,2013-02-15,2013-02-20,2,2,1,11204,3,1,1,2,50,636,0,1,1.0
29634,2013-01-16 21:46:45,2,87.7509,304783,0,2013-01-30,2013-01-31,2,1,1,8834,1,1,1,2,50,355,0,1,1.0
5380,2013-03-07 22:35:43,2,3375.2453,897097,1,2013-04-18,2013-04-23,2,0,1,12257,6,1,1,4,51,1509,0,1,1.0
75396,2013-03-15 23:24:23,2,104.0409,131819,0,2013-03-17,2013-03-18,1,0,1,9147,1,1,1,2,50,367,0,1,1.0
44151,2013-08-04 21:44:44,34,194.0870,511347,0,2013-08-15,2013-08-16,2,2,1,59849,4,1,1,2,198,377,0,1,1.0
44144,2013-09-14 07:05:23,11,4566.3243,548549,0,2013-09-27,2013-10-01,1,1,1,8855,1,1,1,2,50,213,0,1,1.0
911,2013-05-22 03:02:40,2,2758.2586,534936,0,2013-06-14,2013-06-19,2,0,1,669,3,1,1,2,50,212,0,1,1.0
95921,2013-04-15 16:17:51,2,2219.2280,315676,0,2013-04-20,2013-04-27,2,0,1,14127,4,1,1,2,50,682,0,1,1.0
31676,2013-01-07 14:44:36,11,1478.4271,143954,1,2013-02-17,2013-02-24,2,0,1,8791,1,1,1,4,8,110,0,1,1.0
34471,2013-01-18 10:36:13,2,790.0645,575552,0,2013-02-22,2013-02-24,4,1,2,8268,1,1,1,2,50,682,0,1,1.0


In [120]:
train.sort_values("user_id")

,date_time,site_name,orig_destination_distance,user_id,is_package,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,num,ratio
28506,2013-03-28 12:59:33,2,6089.6198,37,1,2013-07-05,2013-07-11,3,0,1,11816,1,1,1,6,77,1482,30,1,1.0
26193,2013-12-23 01:03:25,2,NaN,301,1,2014-01-20,2014-01-24,2,0,1,8791,1,1,1,4,8,110,66,1,1.0
73039,2013-03-28 08:25:34,2,386.0971,591,1,2013-03-30,2013-04-02,1,0,1,8250,1,1,1,2,50,628,88,1,1.0
16083,2013-09-03 11:00:35,2,374.7914,654,0,2013-09-11,2013-09-12,1,0,1,8279,1,1,1,2,50,1230,56,1,1.0
25595,2013-04-02 10:37:54,34,82.2269,706,0,2013-04-13,2013-04-14,1,0,1,44483,1,1,1,2,198,791,48,1,1.0
55076,2013-12-26 06:07:20,34,366.2891,727,0,2014-01-01,2014-01-03,2,4,1,8291,1,1,1,2,50,191,18,1,1.0
66780,2013-07-23 08:19:55,2,224.5950,743,0,2013-07-25,2013-07-26,2,0,1,1194,3,1,1,4,8,123,43,1,1.0
84493,2013-06-28 11:50:54,2,348.0917,752,0,2013-07-02,2013-07-03,2,1,1,12215,6,1,1,2,50,365,10,1,1.0
19716,2013-05-22 16:07:20,2,584.2942,763,0,2013-07-11,2013-07-14,2,0,1,3072,3,1,1,2,50,366,4,1,1.0
82180,2013-10-02 14:45:35,2,152.6103,764,0,2013-11-22,2013-11-24,2,0,1,328,1,1,1,2,50,926,32,1,1.0


In [143]:
train.groupby(['srch_destination_id',
                         'hotel_cluster'])['is_booking'].agg(['sum','count']).sort_values("count", ascending=False)

,,sum,count
srch_destination_id,hotel_cluster,,
8250,1,26519,309484
8791,65,5671,177405
8250,45,13760,177232
8267,56,11444,167721
8250,79,12796,167188
8267,70,6861,125801
11439,65,4145,124353
8250,24,9402,122226
12206,1,9719,121597


In [142]:
train[train["hotel_cluster"] == 3].sort_values("is_booking")

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
741,2,3,66,143,6959,4095.5567,3994,0,0,9,2015-03-11,2015-03-15,2,0,1,267,3,0,1,6,70,1926,3
24823601,24,2,231,69,14045,NaN,208960,0,0,2,2014-07-26,2014-07-28,1,0,1,19940,6,0,1,3,99,88,3
24823613,24,2,231,80,26073,NaN,208960,0,0,1,2014-08-11,2014-08-12,2,0,1,8220,1,0,1,3,182,46,3
24823928,2,3,66,149,47654,1144.0459,209937,0,0,9,2014-07-28,2014-07-31,1,0,1,12615,5,0,2,2,50,676,3
24823929,2,3,66,149,47654,1144.0459,209937,0,0,9,2014-07-28,2014-07-31,2,0,1,12615,5,0,1,2,50,676,3
24823946,2,3,66,348,5757,195.8081,209937,0,0,9,2014-07-31,2014-08-03,1,0,1,7748,3,0,1,2,50,676,3
24824338,2,3,66,254,7813,1706.0148,211375,0,1,9,2014-02-28,2014-03-07,1,2,2,11942,1,0,1,4,193,99,3
24824357,2,3,66,337,16209,NaN,211438,0,0,9,2013-06-26,2013-06-29,2,2,1,12281,6,0,1,2,50,191,3
24824360,2,3,66,337,16209,NaN,211438,0,0,9,2013-06-26,2013-06-29,2,2,1,8291,1,0,1,2,50,191,3
24824423,2,3,66,442,3715,5703.8706,211697,0,0,3,2015-08-22,2015-08-25,2,0,1,18077,1,0,1,6,15,150,3


In [139]:
train[train["hotel_cluster"] == 44]

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
730,25,2,23,48,4924,NaN,3972,1,0,1,2014-11-07,2014-11-10,2,0,1,33452,1,0,1,3,99,1308,44
734,2,3,66,143,6959,294.6852,3994,0,0,3,2014-09-25,2014-09-28,2,0,1,26495,1,0,1,2,50,454,44
880,33,2,194,38,42328,NaN,5776,0,0,2,2014-10-04,2014-10-06,2,0,1,1657,3,0,1,3,151,1189,44
1025,11,3,205,354,25315,1933.3408,6450,0,0,5,2015-03-02,2015-03-09,2,0,1,1442,3,0,2,4,125,177,44
1272,2,3,66,314,46561,928.2655,7041,1,1,9,2014-12-10,2014-12-10,1,0,1,27947,6,0,1,4,128,1400,44
1352,2,3,66,174,17468,2400.5667,7283,0,1,2,2014-11-16,2014-11-20,2,0,1,21059,6,0,1,4,8,110,44
1379,2,3,66,174,27251,2405.8380,7283,0,1,9,2014-11-17,2014-11-22,2,0,1,8791,1,0,1,4,8,110,44
1386,2,3,66,174,42586,2392.1498,7283,0,1,9,2014-11-18,2014-11-22,2,0,1,8791,1,0,1,4,8,110,44
1481,2,3,66,174,21356,5854.3395,7523,1,1,9,2014-11-06,2014-11-09,4,0,1,41121,1,0,2,4,124,1943,44
1676,2,3,66,143,23750,1923.7573,7783,0,1,9,2013-09-13,2013-09-21,2,0,1,9171,1,0,1,4,118,200,44


In [145]:
train[train["srch_destination_id"] == 8250].sort_values("is_booking", ascending=False)

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
21820117,11,3,205,354,48951,1940.5972,1130904,0,0,9,2013-05-20,2013-05-23,3,0,1,8250,1,1,1,2,50,628,1
33250914,2,3,66,462,49272,877.3595,1042877,0,0,9,2014-04-14,2014-04-16,2,3,2,8250,1,1,1,2,50,628,88
12050855,2,3,66,435,5391,1736.7710,629765,0,1,9,2013-06-18,2013-06-23,1,0,1,8250,1,1,1,2,50,628,24
33250970,2,3,66,174,45898,NaN,1043157,0,0,0,2013-12-21,2013-12-22,2,0,1,8250,1,1,1,2,50,628,45
21339313,11,3,205,135,29816,1052.0141,693381,0,1,0,2015-01-24,2015-01-27,2,0,1,8250,1,1,1,2,50,628,1
23246364,37,1,69,749,51392,NaN,344795,0,0,5,2015-01-06,2015-01-10,1,0,1,8250,1,1,2,2,50,628,19
21339254,2,3,66,220,43026,1973.4137,692349,0,1,0,2014-11-18,2014-11-22,2,0,1,8250,1,1,1,2,50,628,79
12050629,2,3,235,13,53632,NaN,629646,0,0,6,2013-07-12,2013-07-15,2,1,1,8250,1,1,1,2,50,628,51
35396011,2,3,66,337,10841,2221.6079,1140829,0,0,9,2014-01-13,2014-01-15,1,0,1,8250,1,1,1,2,50,628,1
12050621,2,3,235,13,53632,NaN,629646,0,0,6,2013-08-25,2013-08-28,2,2,1,8250,1,1,1,2,50,628,1


In [124]:
agg.head()

,srch_destination_id,hotel_cluster,bookings,clicks,relevance
0,4,22,0.0,1.0,0.05
1,4,30,0.0,1.0,0.05
2,4,32,1.0,1.0,1.05
3,4,78,0.0,1.0,0.05
4,4,82,0.0,1.0,0.05
